In [1]:
import cv2
import numpy as np
import os
import math # For angle calculations
import pandas as pd # For CSV output

# --- Configuration: Hardcoded Parameters from Tuning ---
# IMPORTANT: Replace these values with the optimal parameters you found during tuning.
# Example values are provided, but your tuned values will be different.

# L*a*b* Color Space Ranges for Orange Line
LOWER_L_TUNED = 147
UPPER_L_TUNED = 221
LOWER_A_TUNED = 135
UPPER_A_TUNED = 157
LOWER_B_TUNED = 135
UPPER_B_TUNED = 161

# Morphological Kernel Size for Color Mask
COLOR_MORPH_KERNEL_SIZE_TUNED = 3

# Edge Pre-processing Filter (Bilateral or Gaussian)
USE_BILATERAL_FILTER_TUNED = 1 # 0 for Gaussian, 1 for Bilateral
BILATERAL_D_TUNED = 6
BILATERAL_SIGMA_COLOR_TUNED = 7
BILATERAL_SIGMA_SPACE_TUNED = 200

# Canny Edge Detector Thresholds
CANNY_THRESH1_TUNED = 3
CANNY_THRESH2_TUNED = 93

# Morphological Kernel Size for Edge Mask
EDGE_MORPH_KERNEL_SIZE_TUNED = 25

# Hough Line Transform Parameters
HOUGH_THRESHOLD_TUNED =15
HOUGH_MIN_LENGTH_TUNED =160
HOUGH_MAX_GAP_TUNED = 45

# Line Filtering Parameters (based on angle)
MAX_LINE_ANGLE_DEG_TUNED = 88 # Maximum allowed deviation from horizontal (0 or 180 degrees)

# Image Cropping Parameter
CROP_PERCENT_FROM_TOP_TUNED = 57 # Percentage of image height to crop from the top


# --- File Paths ---
# IMPORTANT: ADJUST THIS PATH to the folder containing your Carla environment images.
IMAGE_FOLDER = "C:/Users/BCI-Lab/Downloads/teamA_dataset/_out_dataset/good_data" 
OUTPUT_CSV_FILENAME = "lane_line_features_extracted.csv" # Name of the output CSV file


# --- Function to Process a Single Image and Extract Features ---
def extract_lane_line_features(image_path, params):
    """
    Processes a single image to detect the orange lane line and extract relevant features.

    Args:
        image_path (str): Full path to the image file.
        params (dict): Dictionary containing all the tuned parameters.

    Returns:
        dict: A dictionary of extracted features for the image.
              Returns default/NaN values if no line is detected or on error.
    """
    features = {
        "filename": os.path.basename(image_path),
        "is_line_detected": 0,
        "centroid_x_cropped": -1.0,
        "centroid_y_cropped": -1.0,
        "deviation_score": float('nan'), # Normalized deviation (-1 to 1)
        "num_line_segments": 0,
        "total_line_pixel_length": 0.0,
        "avg_line_angle_deg": 0.0,
        "std_line_angle_deg": 0.0,
        "min_line_y_cropped": -1.0, # Closest to top of cropped image
        "max_line_y_cropped": -1.0, # Closest to bottom of cropped image
        "fitted_line_slope_deg": 0.0,
        "fitted_line_x_intercept_at_bottom_cropped": -1.0,
        "fitted_line_x_intercept_at_top_cropped": -1.0,
    }

    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ ERROR: Cannot read {image_path}. Skipping.")
        return features # Return default features

    original_h, original_w, _ = image.shape
    center_x_full_image = original_w // 2

    # --- 1. Crop from top ---
    crop_y = int(original_h * params["crop_percent_from_top"] / 100)
    if crop_y >= original_h:
        crop_y = original_h - 1
    cropped_image = image[crop_y:, :].copy()
    cropped_h, cropped_w, _ = cropped_image.shape

    # --- 2. Color Masking (L*a*b*) ---
    lab_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_cropped)
    
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l_eq = clahe.apply(l_channel)
    lab_eq = cv2.merge([l_eq, a_channel, b_channel])
    
    blurred_lab_eq = cv2.medianBlur(lab_eq, 5) 

    lower_orange_lab = np.array([params["lower_l"], params["lower_a"], params["lower_b"]])
    upper_orange_lab = np.array([params["upper_l"], params["upper_a"], params["upper_b"]])
    color_mask = cv2.inRange(blurred_lab_eq, lower_orange_lab, upper_orange_lab)

    # --- 3. Morphological Operations on Color Mask ---
    color_morph_kernel = np.ones((params["color_morph_kernel_size"], params["color_morph_kernel_size"]), np.uint8)
    color_mask_morphed = cv2.morphologyEx(color_mask, cv2.MORPH_OPEN, color_morph_kernel, iterations=1)
    color_mask_morphed = cv2.dilate(color_mask_morphed, color_morph_kernel, iterations=1) 
    color_mask_morphed = cv2.morphologyEx(color_mask_morphed, cv2.MORPH_CLOSE, color_morph_kernel, iterations=1)
    
    # --- 4. Edge Detection (Canny) ---
    gray_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    
    if params["use_bilateral_filter"]:
        filtered_gray = cv2.bilateralFilter(gray_cropped, params["bilateral_d"], 
                                             params["bilateral_sigma_color"], params["bilateral_sigma_space"])
    else:
        filtered_gray = cv2.GaussianBlur(gray_cropped, (5, 5), 0)

    edge_mask = cv2.Canny(filtered_gray, params["canny_thresh1"], params["canny_thresh2"])

    edge_morph_kernel = np.ones((params["edge_morph_kernel_size"], params["edge_morph_kernel_size"]), np.uint8)
    edge_mask_morphed = cv2.morphologyEx(edge_mask, cv2.MORPH_CLOSE, edge_morph_kernel, iterations=1)

    # --- 5. Combine Color Mask and Morphed Edge Mask ---
    final_mask = cv2.bitwise_and(color_mask_morphed, edge_mask_morphed)

    # --- 6. Line Detection with Hough Transform ---
    lines = cv2.HoughLinesP(final_mask, 1, np.pi / 180, 
                            params["hough_threshold"], 
                            minLineLength=params["hough_min_length"], 
                            maxLineGap=params["hough_max_gap"])

    # --- 7. Filter Detected Lines and Calculate Features ---
    filtered_lines = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angle_rad = math.atan2(y2 - y1, x2 - x1)
            angle_deg = math.degrees(angle_rad)

            if abs(angle_deg) < params["max_line_angle_deg"] or abs(angle_deg - 180) < params["max_line_angle_deg"]:
                filtered_lines.append(line[0]) # Store as flat list [x1, y1, x2, y2]

    if filtered_lines:
        features["is_line_detected"] = 1
        all_x_coords = []
        all_y_coords = []
        all_line_angles = []
        all_line_lengths = []

        for line_seg in filtered_lines:
            x1, y1, x2, y2 = line_seg
            all_x_coords.extend([x1, x2])
            all_y_coords.extend([y1, y2])

            angle_rad = math.atan2(y2 - y1, x2 - x1)
            all_line_angles.append(math.degrees(angle_rad))
            
            length = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            all_line_lengths.append(length)
        
        # Calculate Centroid (mean of all segment points)
        features["centroid_x_cropped"] = np.mean(all_x_coords)
        features["centroid_y_cropped"] = np.mean(all_y_coords)

        # Deviation Score (normalized to full image center)
        # Convert centroid_x_cropped to full image x-coord first
        centroid_x_full_image_coord = features["centroid_x_cropped"] 
        if center_x_full_image != 0:
            features["deviation_score"] = (centroid_x_full_image_coord - center_x_full_image) / center_x_full_image
        else:
            features["deviation_score"] = 0.0 # Should not happen with valid images

        # Number of Line Segments
        features["num_line_segments"] = len(filtered_lines)

        # Total Line Pixel Length
        features["total_line_pixel_length"] = np.sum(all_line_lengths)

        # Average and Standard Deviation of Line Angles
        if all_line_angles:
            features["avg_line_angle_deg"] = np.mean(all_line_angles)
            if len(all_line_angles) > 1: # std dev requires at least 2 points
                features["std_line_angle_deg"] = np.std(all_line_angles)
            else:
                features["std_line_angle_deg"] = 0.0

        # Min and Max Y-coordinates (closest to top/bottom of cropped region)
        if all_y_coords:
            features["min_line_y_cropped"] = np.min(all_y_coords)
            features["max_line_y_cropped"] = np.max(all_y_coords)

        # Fitted Line Slope and Intercepts (y = mx + c)
        if len(all_x_coords) > 1 and len(np.unique(all_x_coords)) > 1: # Need at least 2 distinct X points for a slope
            try:
                slope, intercept = np.polyfit(all_x_coords, all_y_coords, 1)
                features["fitted_line_slope_deg"] = math.degrees(math.atan(slope))

                # Calculate x-intercepts at top and bottom of cropped image
                # y = mx + c  => x = (y - c) / m
                if slope != 0:
                    features["fitted_line_x_intercept_at_bottom_cropped"] = (cropped_h - 1 - intercept) / slope
                    features["fitted_line_x_intercept_at_top_cropped"] = (0 - intercept) / slope
                else: # Horizontal line, x-intercepts are undefined/at infinity for slope=0
                    features["fitted_line_x_intercept_at_bottom_cropped"] = features["centroid_x_cropped"]
                    features["fitted_line_x_intercept_at_top_cropped"] = features["centroid_x_cropped"]
            except np.linalg.LinAlgError:
                # This can happen if all points are vertically aligned (slope is infinite)
                # In such cases, the slope will be reported as 90 degrees or -90 degrees
                # and intercepts will be effectively the same x-coordinate.
                features["fitted_line_slope_deg"] = 90.0 if np.mean(all_y_coords) < cropped_h/2 else -90.0
                features["fitted_line_x_intercept_at_bottom_cropped"] = features["centroid_x_cropped"]
                features["fitted_line_x_intercept_at_top_cropped"] = features["centroid_x_cropped"]
            except Exception as e:
                print(f"Warning: Error fitting line for {image_path}: {e}")

    return features

# --- Main Execution Block ---
if __name__ == "__main__":
    print("--- Starting Lane Line Feature Extraction ---")

    # Collect image file paths
    image_files = [f for f in os.listdir(IMAGE_FOLDER) if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff"))]
    if not image_files:
        print(f"ERROR: No images found in '{IMAGE_FOLDER}'. Please check the path.")
        exit()

    print(f"Found {len(image_files)} images in '{IMAGE_FOLDER}'.")

    # Store all tuned parameters in a dictionary
    tuned_parameters = {
        "lower_l": LOWER_L_TUNED,
        "upper_l": UPPER_L_TUNED,
        "lower_a": LOWER_A_TUNED,
        "upper_a": UPPER_A_TUNED,
        "lower_b": LOWER_B_TUNED,
        "upper_b": UPPER_B_TUNED,
        "color_morph_kernel_size": COLOR_MORPH_KERNEL_SIZE_TUNED,
        "use_bilateral_filter": USE_BILATERAL_FILTER_TUNED,
        "bilateral_d": BILATERAL_D_TUNED,
        "bilateral_sigma_color": BILATERAL_SIGMA_COLOR_TUNED,
        "bilateral_sigma_space": BILATERAL_SIGMA_SPACE_TUNED,
        "canny_thresh1": CANNY_THRESH1_TUNED,
        "canny_thresh2": CANNY_THRESH2_TUNED,
        "edge_morph_kernel_size": EDGE_MORPH_KERNEL_SIZE_TUNED,
        "hough_threshold": HOUGH_THRESHOLD_TUNED,
        "hough_min_length": HOUGH_MIN_LENGTH_TUNED,
        "hough_max_gap": HOUGH_MAX_GAP_TUNED,
        "max_line_angle_deg": MAX_LINE_ANGLE_DEG_TUNED,
        "crop_percent_from_top": CROP_PERCENT_FROM_TOP_TUNED,
    }

    all_extracted_features = []
    
    for i, filename in enumerate(image_files):
        full_image_path = os.path.join(IMAGE_FOLDER, filename)
        print(f"Processing image {i+1}/{len(image_files)}: {filename}")
        
        features = extract_lane_line_features(full_image_path, tuned_parameters)
        all_extracted_features.append(features)

    # Create a pandas DataFrame from the collected features
    df = pd.DataFrame(all_extracted_features)

    # Save the DataFrame to a CSV file
    output_csv_path = os.path.join(os.path.dirname(IMAGE_FOLDER), OUTPUT_CSV_FILENAME)
    df.to_csv(output_csv_path, index=False)

    print(f"\n✅ Feature extraction complete! Data saved to: {output_csv_path}")

--- Starting Lane Line Feature Extraction ---
Found 1291 images in 'C:/Users/BCI-Lab/Downloads/teamA_dataset/_out_dataset/good_data'.
Processing image 1/1291: 00420478.png
Processing image 2/1291: 00420523.png
Processing image 3/1291: 00420564.png
Processing image 4/1291: 00420604.png
Processing image 5/1291: 00420644.png
Processing image 6/1291: 00420684.png
Processing image 7/1291: 00420725.png
Processing image 8/1291: 00420765.png
Processing image 9/1291: 00420803.png
Processing image 10/1291: 00420841.png
Processing image 11/1291: 00420878.png
Processing image 12/1291: 00420915.png
Processing image 13/1291: 00420952.png
Processing image 14/1291: 00420989.png
Processing image 15/1291: 00421026.png
Processing image 16/1291: 00421063.png
Processing image 17/1291: 00421100.png
Processing image 18/1291: 00421137.png
Processing image 19/1291: 00421175.png
Processing image 20/1291: 00421214.png
Processing image 21/1291: 00421255.png
Processing image 22/1291: 00421296.png
Processing image 